<a href="https://colab.research.google.com/github/joaovctrr/pdf_to_excel/blob/main/ler_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
# !pip install tabula-py

In [128]:
import tabula
import pandas as pd
import os
import re

In [ ]:
pdf = tabula.read_pdf('arquivo.pdf', pages = 'all')

In [ ]:
pdf

In [ ]:
print(type(pdf))

In [104]:
df = pdf[2]

In [ ]:
df

In [106]:
df_completo = pd.concat(pdf, ignore_index=False)


In [ ]:
df_completo

In [108]:
df_completo.columns = df_completo.columns.str.replace('\r', ' ')

for col in df_completo.columns:
    if df_completo[col].dtype == 'object':
        df_completo[col] = df_completo[col].str.replace('\r', ' ')


In [ ]:
df_completo

In [110]:
df = df_completo

In [ ]:
print(df['Unnamed: 0'])


In [112]:
df['Unnamed: 0'] = df['Unnamed: 0'].astype(str)

In [ ]:
df['Unnamed: 0']

In [ ]:
dados_capturados

In [133]:
diretorio_pdfs = '.'


nomes_pdfs = []

for filename in os.listdir(diretorio_pdfs):
    if filename.endswith('.pdf'):
        nomes_pdfs.append(os.path.join(diretorio_pdfs, filename))

dfs = []

for pdf_file in nomes_pdfs:
    try:
                df = tabula.read_pdf(pdf_file, pages='all', pandas_options={'header': None})
        dfs.extend(df)
    except Exception as e:
        print(f"Erro ao processar o arquivo {pdf_file}: {str(e)}")

df_final = pd.concat(dfs, ignore_index=True)

# Inicializa listas para armazenar as informações
nomes = []
locais = []
numeros = []
municipios = []
pontos_referencia = []

# Variáveis auxiliares para acompanhar o nome completo e o número atual
nome_atual = ''
numero_atual = ''

# Expressão regular para identificar números de endereço no formato XX ou XXXX (dois ou quatro dígitos)
regex_numero = re.compile(r'\b\d{2,4}\b')

for index, row in df_final.iterrows():
    if isinstance(row[0], str):
        if 'PONTO DE REFERÊNCIA' in row[0]:
            partes = row[0].split('PONTO DE REFERÊNCIA')
            if len(partes) > 1:
                pontos_referencia.append(partes[1].strip())
                if nome_atual:
                    nomes.append(nome_atual.strip())
                nome_atual = ''
        elif 'LOCAL (AV., RUA, ETC)' in row[0]:
            partes = row[0].split('LOCAL (AV., RUA, ETC)')
            if len(partes) > 1:
                locais.append(partes[1].strip())
        elif 'NÚMERO' in row[0]:
            partes = row[0].split('NÚMERO')
            if len(partes) > 1:
                numero = partes[1].strip()
                # Verifica se o número corresponde ao padrão de endereço esperado
                if regex_numero.match(numero):
                    numeros.append(numero)
                    if nome_atual:
                        nomes.append(nome_atual.strip())
                else:
                    numero_atual = numero
                nome_atual = ''
        elif 'MUNICÍPIO' in row[0]:
            partes = row[0].split('MUNICÍPIO')
            if len(partes) > 1:
                municipios.append(partes[1].strip())
        elif 'NOME COMPLETO' in row[0]:
            partes = row[0].split('NOME COMPLETO')
            if len(partes) > 1:
                if nome_atual:
                    nome_atual += ', ' + partes[1].strip()
                else:
                    nome_atual = partes[1].strip()


if nome_atual:
    nomes.append(nome_atual.strip())
    if not numeros:
        numeros.append(numero_atual)

max_length = max(len(nomes), len(locais), len(numeros), len(municipios), len(pontos_referencia))

# Preenche as listas com valores vazios para terem o mesmo comprimento
nomes += [''] * (max_length - len(nomes))
locais += [''] * (max_length - len(locais))
numeros += [''] * (max_length - len(numeros))
municipios += [''] * (max_length - len(municipios))
pontos_referencia += [''] * (max_length - len(pontos_referencia))

# Cria um DataFrame com as informações capturadas
dados_capturados = pd.DataFrame({
    'NOME COMPLETO': nomes,
    'LOCAL': locais,
    'NÚMERO': numeros,
    'MUNICÍPIO': municipios,
    'PONTO DE REFERÊNCIA': pontos_referencia
})

nome_excel = './dados_extraidos.xlsx'

# Salva o DataFrame em um arquivo Excel
dados_capturados.to_excel(nome_excel, index=False, engine='openpyxl')

print(f'Dados salvos com sucesso em {nome_excel}')


Jun 14, 2024 7:21:17 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

Jun 14, 2024 7:21:20 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

Jun 14, 2024 7:21:23 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

Jun 14, 2024 7:21:26 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

Jun 14, 2024 7:21:29 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



Dados salvos com sucesso em ./dados_extraidos.xlsx
